### **Importación dataset y librerías**

In [ ]:
# Manipulación y visualización
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.svm import SVC

# Preprocesamiento
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Métrica de evaluación
from sklearn.metrics import accuracy_score

# Feature Selection
from sklearn.feature_selection import SelectKBest, f_classif, RFE

# Modificar máximo de columnas
pd.set_option('display.max_columns', None)

### **Importación del Dataset**

In [ ]:
data = pd.read_csv('high_diamond_ranked_10min.csv')

### **Preprocesamiento del laboratorio anterior**

In [ ]:
# Excluir columnas innecesarias
df = data.drop(["gameId","blueGoldPerMin","redGoldPerMin"], axis=1)

# Separar la variable objetivo
np.set_printoptions(suppress=True)
Xcolumns = df.iloc[:, 1:].columns.values
X = df.iloc[:, 1:].values
Y = df.iloc[:, 0].values

# Data Split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=48)
X_val, X_test, Y_val, Y_test = train_test_split(X_test, Y_test, test_size=0.5, random_state=48)

# Escalado de Variables
X_train_df = pd.DataFrame(X_train, columns=Xcolumns)
X_test_df = pd.DataFrame(X_test, columns=Xcolumns)
X_val_df = pd.DataFrame(X_val, columns=Xcolumns)

sc_X = StandardScaler()
X_train_df = pd.DataFrame(sc_X.fit_transform(X_train_df), columns=Xcolumns)
X_test_df = pd.DataFrame(sc_X.transform(X_test_df), columns=Xcolumns)
X_val_df = pd.DataFrame(sc_X.transform(X_val_df), columns=Xcolumns)

#### Técnicas de Feature Selection:

In [ ]:
# Técnica 1: Correlación de Pearson
correlaciones = X_train_df.corrwith(pd.Series(Y_train)).abs().sort_values(ascending=False)
features_pearson = correlaciones.head(10).index.tolist()  # Top 10

# Técnica 2: ANOVA F-value
selector_anova = SelectKBest(score_func=f_classif, k=10)
X_train_anova = selector_anova.fit_transform(X_train_df, Y_train)
features_anova = X_train_df.columns[selector_anova.get_support()].tolist()

# Técnica 3: RFE con SVM
svm_rfe = SVC(kernel='linear', C=1)
selector_rfe = RFE(svm_rfe, n_features_to_select=10)
selector_rfe.fit(X_train_df, Y_train)
features_rfe = X_train_df.columns[selector_rfe.support_].tolist()

In [ ]:
# Función para entrenar y evaluar
def evaluar_modelo(features):
    print("Features a evaluar:\n", features)
    X_train_sel = X_train_df[features]
    X_test_sel = X_test_df[features]
    
    model = SVC(kernel='linear', C=1)
    model.fit(X_train_sel, Y_train)
    
    y_pred = model.predict(X_test_sel)
    acc = accuracy_score(Y_test, y_pred)
    print(acc)
    return acc

accuracy_original = 0.7206  # Obtenido en el laboratorio anterior

# Accuracy con cada técnica
accuracy_pearson = evaluar_modelo(features_pearson)
accuracy_anova = evaluar_modelo(features_anova)
accuracy_rfe = evaluar_modelo(features_rfe)

print("------------ Resumen Final ------------")
print("Accuracy Original:\n",accuracy_original)
print("Accuracy Pearson:\n",accuracy_pearson)
print("Accuracy Anova:\n",accuracy_anova)
print("Accuracy RFE:\n",accuracy_rfe)

Features a evaluar:
 ['redGoldDiff', 'blueGoldDiff', 'redExperienceDiff', 'blueExperienceDiff', 'blueTotalGold', 'redTotalGold', 'blueTotalExperience', 'redTotalExperience', 'blueAvgLevel', 'redAvgLevel']
0.7236842105263158
Features a evaluar:
 ['blueTotalGold', 'blueAvgLevel', 'blueTotalExperience', 'blueGoldDiff', 'blueExperienceDiff', 'redTotalGold', 'redAvgLevel', 'redTotalExperience', 'redGoldDiff', 'redExperienceDiff']
0.7236842105263158
Features a evaluar:
 ['blueDragons', 'blueTotalGold', 'blueTotalExperience', 'blueGoldDiff', 'blueExperienceDiff', 'redDragons', 'redTotalGold', 'redTotalExperience', 'redGoldDiff', 'redExperienceDiff']
0.7246963562753036
------------ Resumen Final ------------
Accuracy Original:
 0.7206
Accuracy Pearson:
 0.7236842105263158
Accuracy Anova:
 0.7236842105263158
Accuracy RFE:
 0.7246963562753036


#### **Análisis de Resultados**
A continuación se presentaran las diferencias de las técnicas implementadas:

<center>

|    Técnica    |    Accuracy   | Num. Features |     Observación     |
| ------------- | ------------- |-------------- | ------------------- |
| **Original**  |    72.06%     |       37      | Se utilizaron todas las features |
| **Pearson**   |    72.37%     |       10      | Incluye Features de diferencia de puntuaciones en la partida |
| **ANOVA**     |    72.37%     |       10      | Mismas Features que **Pearson**  |
| **RFE**       |    72.47%     |       10      | Incluye Features de los objetivos tomados por los equipos|

</center>

* RFE utiliza el propio SVM para evaluar la importancia de las características, capturando iteraciones no lineales entre variables. Mientras que Pearson y ANOVA solo miden las relaciones individuales con ```blueWins```
* Como se menciona en la tabla, ```RFE``` incluyó las features que describen las ventajas obtenidas durante el juego, mientras que ```Pearson``` y ```ANOVA``` se enfocaron más en describir las diferencias de estadísticas de economía o puntos de experiencia en el juego.

#### Comparación directa
* Pearson vs ANOVA:
    * Ambas técnicas dieron el mismo accuracy (72.37%), debido a que seleccionaron las mismas features. Esto sugiere que las diferencias de oro y experiencia entre los equipos son críticas para predecir qué equipo gana, pero al mismo tiempo demuestra que no aportan información nueva al incluirse todas juntas.
    * RFE vs Original:
        * El modelo con RFE superó al original por **0.41%** utilizando **27** features menos. Lo cuál demuestra que la mayoría de las 37 caracterísitcas originales eran redundantes o irrelevantes.

### **Conclusión Final**
En este caso es recomendable utilizar el método de **RFE** ya que nos brindó un mejor accuracy y mejor eficiencia comparado al modelo original.
Además las features seleccionadas demuestran la importancia de ciertos objetivos en el juego que le brindan ciertas ventajas al equipo que los alcanza.